In [ ]:
#load packages
import stereo as st
import pandas as pd
import numpy as np
import scanpy as sc
import anndata as an
import matplotlib.pyplot as plt
from matplotlib import colormaps as cmaps
plt.figure(dpi=300)
import warnings
warnings.filterwarnings('ignore')
from tqdm.notebook import tqdm

import os as os
#verify proper environment is being used on this server
import sys
print(sys.version)#should be python 3.8
import sys
print(sys.executable)#should be stereopy_env in miniconda parent directory

In [ ]:
#load unfiltered stereo-seq data
data = st.io.read_h5ad(file_path='/home/ryan/Sean/Object/StereoSeq/PigP10_bin20.stereo.h5ad', flavor='stereopy')
data#view stereopy object prepared from stereoseq
#adata = st.io.read_h5ad(file_path='/home/ryan/Sean/Object/StereoSeq/PigP10_bin20.stereo.forScanpy.h5ad', flavor='scanpy')
#adata#view adata object prepared for scanpy from stereoseq

In [ ]:
#visualize QC in aggregate
data.plt.violin()
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/1_QC_Violin.png', dpi=300)
data.plt.genes_count()
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/2_QC_Scatter.png', dpi=300)

In [ ]:
#Visualize QC on the spatial tissue map
data.plt.spatial_scatter(out_dpi=500)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/3_QC_SpatialScatter.png', dpi=500)
data.plt.spatial_scatter(base_image='/home/ryan/Sean/IMG/PigP10_crop/B03701E2_ssDNA_regist.tif', base_im_to_gray=True, out_dpi=500)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/3_QC_SpatialScatter_wMask.png', dpi=500)

In [ ]:
#do QC filtration
data
#min_n_genes_by_counts: minimum number of total counts required for a cell to pass fitlering
#min_gene: minimum number of genes expressed required for a cell to pass filtering
data.tl.filter_cells(
        min_n_genes_by_counts=50,
        min_gene=5,
        pct_counts_mt=10,
        inplace=True
        )#in ~stereopy/stereo/core/st_pipeline.py:162, filter_cells is defined with parameters 'min_gene'/'max_gene'/'min_n_genes_by_counts' not 'min_genes'/'max_genes'/'min_counts' like in stereopy vignette
data

#Visualize QC on the spatial tissue map
data.plt.spatial_scatter(out_dpi=500)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/4_QC_Filtered_SpatialScatter.png', dpi=500)
data.plt.spatial_scatter(base_image='/home/ryan/Sean/IMG/PigP10_crop/B03701E2_ssDNA_regist.tif', base_im_to_gray=True, out_dpi=500)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/4_QC_Filtered_SpatialScatter_wMask.png', dpi=500)

In [ ]:
data.tl.raw_checkpoint()#save the stereopy data object before normalizing
data.tl.raw
#data.tl.reset_raw_data()#restore data object to the saved raw data object

In [ ]:
#use scTransform to normalize data and identify variable genes
data.tl.sctransform(res_key='sctransform', inplace=True, filter_hvgs=False, n_cells=10000, n_genes=2000)
#https://stereopy.readthedocs.io/en/latest/Tutorials/scTransform.html

In [ ]:
# run PCA and identify number of PCs to use for clustering
data.tl.pca(use_highly_genes=False, hvg_res_key='highly_variable_genes', n_pcs=30, res_key='pca', svd_solver='arpack')
data.plt.elbow(pca_res_key='pca')
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/5_PC_ElbowPlot.png', dpi=300)

In [ ]:
data.tl.neighbors(pca_res_key='pca', n_pcs=20, res_key='neighbors', n_jobs=6)
data.tl.umap(pca_res_key='pca', neighbors_res_key='neighbors', res_key='umap', init_pos='spectral', spread=2.0)

In [ ]:
#visualize expression of marker genes across the UMAP
data.plt.umap(gene_names=['KRT15', 'KRT10'], res_key='umap', out_dpi=500)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/6_KRT15_KRT10_UMAP.png', dpi=500)

data.plt.umap(gene_names=['COL1A1','COL3A1'], res_key='umap', out_dpi=500)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/6_COL1A1_COL3A1_UMAP.png', dpi=500)

In [ ]:
#Leiden clustering
data.tl.leiden(neighbors_res_key='neighbors',res_key='leiden', resolution=1)#resolution=1 is default

In [ ]:
#view Leiden clustering onto the mask of the tissue section
data.plt.cluster_scatter(res_key='leiden')
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/7_Spatial_Clustering.png', dpi=500)
data.plt.cluster_scatter(res_key='leiden', base_image='/home/ryan/Sean/IMG/PigP10_crop/B03701E2_ssDNA_regist.tif', foreground_alpha=1)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/7_Spatial_Clustering_wMask.png', dpi=500)

In [ ]:
#view subset of Leiden clustering onto the mask of the tissue section
data.plt.cluster_scatter(res_key='leiden', groups=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
                                                  '12', '13', '14', '15', '16', '17', 
                                                   '18', '19', '20', '21', '22', '23'], 
                         others_color='#FFFFFF', foreground_alpha=1)#exclude cluster 11 (not tissue)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/8_Spatial_Clustering_focus.png', dpi=500)
data.plt.cluster_scatter(res_key='leiden', groups=['16', '18'], base_image='/home/ryan/Sean/IMG/PigP10_crop/B03701E2_ssDNA_regist.tif',
                        others_color='#FFFFFF', foreground_alpha=1)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/8_filtered_Spatial_Clustering_focus_Epi_wMask.png', dpi=500)

In [ ]:
data.plt.umap(res_key='umap', cluster_key='leiden')
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/9_Leiden_UMAP.png', dpi=500)

In [ ]:
#view spatial gene expression across the tissue mask
data.plt.spatial_scatter_by_gene(gene_name='KRT14', palette='linear_grey_10_95_c0',out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/KRT14_Spatial_Expression(grey).png', dpi=500)
data.plt.spatial_scatter_by_gene(gene_name='KRT15', palette='linear_grey_10_95_c0',out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/KRT15_Spatial_Expression(grey).png', dpi=500)
data.plt.spatial_scatter_by_gene(gene_name='KRT10', palette='linear_grey_10_95_c0',out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/KRT10_Spatial_Expression(grey).png', dpi=500)
data.plt.spatial_scatter_by_gene(gene_name='COL1A1', palette='linear_grey_10_95_c0',out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/COL1A1_Spatial_Expression(grey).png', dpi=500)
data.plt.spatial_scatter_by_gene(gene_name='COL3A1', palette='linear_grey_10_95_c0',out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/COL3A1_Spatial_Expression(grey).png', dpi=500)
data.plt.spatial_scatter_by_gene(gene_name='PDGFRA', palette='linear_grey_10_95_c0',out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/PDGFRA_Spatial_Expression(grey).png', dpi=500)
data.plt.spatial_scatter_by_gene(gene_name='PIEZO2', palette='linear_grey_10_95_c0',out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/PIEZO2_Spatial_Expression(grey).png', dpi=500)
data.plt.spatial_scatter_by_gene(gene_name='MFAP5', palette='linear_grey_10_95_c0',out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/MFAP5_Spatial_Expression(grey).png', dpi=500)
data.plt.spatial_scatter_by_gene(gene_name='VIM', palette='linear_grey_10_95_c0',out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/VIM_Spatial_Expression(grey).png', dpi=500)
data.plt.spatial_scatter_by_gene(gene_name='SOX9', palette='linear_grey_10_95_c0',out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/SOX9_Spatial_Expression(grey).png', dpi=500)
data.plt.spatial_scatter_by_gene(gene_name='VEGFA', palette='linear_grey_10_95_c0',out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/VEGFA_Spatial_Expression(grey).png', dpi=500)
data.plt.spatial_scatter_by_gene(gene_name='PDGFC', palette='linear_grey_10_95_c0',out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/PDGFC_Spatial_Expression(grey).png', dpi=500)
data.plt.spatial_scatter_by_gene(gene_name='PECAM1', palette='linear_grey_10_95_c0',out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/PECAM1_Spatial_Expression(grey).png', dpi=500)
data.plt.spatial_scatter_by_gene(gene_name='CDH5', palette='linear_grey_10_95_c0',out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/CDH5_Spatial_Expression(grey).png', dpi=500)
data.plt.spatial_scatter_by_gene(gene_name='ACTA2', palette='linear_grey_10_95_c0',out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/ACTA2_Spatial_Expression(grey).png', dpi=500)
data.plt.spatial_scatter_by_gene(gene_name='RGS5', palette='linear_grey_10_95_c0',out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/RGS5_Spatial_Expression(grey).png', dpi=500)

#view spatial gene expression across the tissue mask
data.plt.spatial_scatter_by_gene(gene_name='KRT14', palette='CET_L4', color_bar_reverse=False, out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/KRT14_Spatial_Expression(CET_L4).png', dpi=500)
data.plt.spatial_scatter_by_gene(gene_name='KRT15', palette='CET_L4', color_bar_reverse=False, out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/KRT15_Spatial_Expression(CET_L4).png', dpi=500)
data.plt.spatial_scatter_by_gene(gene_name='KRT10', palette='CET_L4', color_bar_reverse=False, out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/KRT10_Spatial_Expression(CET_L4).png', dpi=500)
data.plt.spatial_scatter_by_gene(gene_name='COL1A1', palette='CET_L4', color_bar_reverse=False, out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/COL1A1_Spatial_Expression(CET_L4).png', dpi=500)
data.plt.spatial_scatter_by_gene(gene_name='COL3A1', palette='CET_L4', color_bar_reverse=False, out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/COL3A1_Spatial_Expression(CET_L4).png', dpi=500)
data.plt.spatial_scatter_by_gene(gene_name='PDGFRA', palette='CET_L4', color_bar_reverse=False, out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/PDGFRA_Spatial_Expression(CET_L4).png', dpi=500)
data.plt.spatial_scatter_by_gene(gene_name='PIEZO2', palette='CET_L4', color_bar_reverse=False, out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/PIEZO2_Spatial_Expression(CET_L4).png', dpi=500)
data.plt.spatial_scatter_by_gene(gene_name='MFAP5', palette='CET_L4', color_bar_reverse=False, out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/MFAP5_Spatial_Expression(CET_L4).png', dpi=500)
data.plt.spatial_scatter_by_gene(gene_name='VIM', palette='CET_L4', color_bar_reverse=False, out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/VIM_Spatial_Expression(CET_L4).png', dpi=500)
data.plt.spatial_scatter_by_gene(gene_name='SOX9', palette='CET_L4', color_bar_reverse=False, out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/SOX9_Spatial_Expression(CET_L4).png', dpi=500)
data.plt.spatial_scatter_by_gene(gene_name='VEGFA', palette='CET_L4', color_bar_reverse=False, out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/VEGFA_Spatial_Expression(CET_L4).png', dpi=500)
data.plt.spatial_scatter_by_gene(gene_name='PDGFC', palette='CET_L4', color_bar_reverse=False, out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/PDGFC_Spatial_Expression(CET_L4).png', dpi=500)
data.plt.spatial_scatter_by_gene(gene_name='PECAM1', palette='CET_L4', color_bar_reverse=False, out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/PECAM1_Spatial_Expression(CET_L4).png', dpi=500)
data.plt.spatial_scatter_by_gene(gene_name='CDH5', palette='CET_L4', color_bar_reverse=False, out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/CDH5_Spatial_Expression(CET_L4).png', dpi=500)
data.plt.spatial_scatter_by_gene(gene_name='ACTA2', palette='CET_L4', color_bar_reverse=False, out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/ACTA2_Spatial_Expression(CET_L4).png', dpi=500)
data.plt.spatial_scatter_by_gene(gene_name='RGS5', palette='CET_L4', color_bar_reverse=False, out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/RGS5_Spatial_Expression(CET_L4).png', dpi=500)

In [ ]:
#Leiden clustering optimization
#0.8 (1.0 default)
data.tl.leiden(neighbors_res_key='neighbors',res_key='leiden', resolution=0.8)#resolution=1 is default
#view Leiden clustering onto the mask of the tissue section
data.plt.cluster_scatter(res_key='leiden')
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/10_Spatial_Clustering_res0.8.png', dpi=500)
data.plt.cluster_scatter(res_key='leiden', base_image='/home/ryan/Sean/IMG/PigP10_crop/B03701E2_ssDNA_regist.tif', foreground_alpha=1)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/10_Spatial_Clustering_wMask_res0.8.png', dpi=500)
data.plt.umap(res_key='umap', cluster_key='leiden')
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/10_Leiden_UMAP_res0.8.png', dpi=500)

#0.6 (1.0 default)
data.tl.leiden(neighbors_res_key='neighbors',res_key='leiden', resolution=0.6)#resolution=1 is default
#view Leiden clustering onto the mask of the tissue section
data.plt.cluster_scatter(res_key='leiden')
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/11_Spatial_Clustering_res0.6.png', dpi=500)
data.plt.cluster_scatter(res_key='leiden', base_image='/home/ryan/Sean/IMG/PigP10_crop/B03701E2_ssDNA_regist.tif', foreground_alpha=1)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/11_Spatial_Clustering_wMask_res0.6.png', dpi=500)
data.plt.umap(res_key='umap', cluster_key='leiden')
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/11_Leiden_UMAP_res0.8.png', dpi=500)

#0.4 (1.0 default)
data.tl.leiden(neighbors_res_key='neighbors',res_key='leiden', resolution=0.4)#resolution=1 is default
#view Leiden clustering onto the mask of the tissue section
data.plt.cluster_scatter(res_key='leiden')
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/12_Spatial_Clustering_res0.4.png', dpi=500)
data.plt.cluster_scatter(res_key='leiden', base_image='/home/ryan/Sean/IMG/PigP10_crop/B03701E2_ssDNA_regist.tif', foreground_alpha=1)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/12_Spatial_Clustering_wMask_res0.4.png', dpi=500)
data.plt.umap(res_key='umap', cluster_key='leiden')
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/12_Leiden_UMAP_res0.4.png', dpi=500)

#1.2 (1. default)
data.tl.leiden(neighbors_res_key='neighbors',res_key='leiden', resolution=1.2)#resolution=1 is default
#view Leiden clustering onto the mask of the tissue section
data.plt.cluster_scatter(res_key='leiden')
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/13_Spatial_Clustering_res1.2.png', dpi=500)
data.plt.cluster_scatter(res_key='leiden', base_image='/home/ryan/Sean/IMG/PigP10_crop/B03701E2_ssDNA_regist.tif', foreground_alpha=1)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/13_Spatial_Clustering_wMask_res1.2.png', dpi=500)
data.plt.umap(res_key='umap', cluster_key='leiden')
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/13_Leiden_UMAP_res1.2.png', dpi=500)

#1.4 (1.0 default)
data.tl.leiden(neighbors_res_key='neighbors',res_key='leiden', resolution=1.4)#resolution=1 is default
#view Leiden clustering onto the mask of the tissue section
data.plt.cluster_scatter(res_key='leiden')
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/14_Spatial_Clustering_res1.4.png', dpi=500)
data.plt.cluster_scatter(res_key='leiden', base_image='/home/ryan/Sean/IMG/PigP10_crop/B03701E2_ssDNA_regist.tif', foreground_alpha=1)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/14_Spatial_Clustering_wMask_res1.4.png', dpi=500)
data.plt.umap(res_key='umap', cluster_key='leiden')
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/14_Leiden_UMAP_res1.4.png', dpi=500)


In [ ]:
#Leiden clustering (finalized)
data.tl.leiden(neighbors_res_key='neighbors',res_key='leiden', resolution=1)#resolution=1 is default

#view Leiden clustering onto the mask of the tissue section
data.plt.cluster_scatter(res_key='leiden')
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/15_Spatial_Clustering.png', dpi=500)

#view spatial gene expression across the tissue mask
data.plt.cluster_scatter(res_key='leiden', base_image='/home/ryan/Sean/IMG/PigP10_crop/B03701E2_ssDNA_regist.tif', 
                         others_color='#FFFFFF', foreground_alpha=1)#ignore off-tissue cluster (set to white by excluding from in-groups list)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/15_Spatial_Clustering_wMask.png', dpi=500)

data.plt.cluster_scatter(res_key='leiden', groups=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
                                                  '12', '13', '14', '15', '16', '17', 
                                                   '18', '19', '20', '21', '22', '23'], 
                         others_color='#FFFFFF', foreground_alpha=1,
                        color_bar_reverse=False)#exclude cluster 11 (not tissue)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/15_Spatial_Clustering_sub.png', dpi=500)

#view spatial gene expression across the tissue mask
data.plt.cluster_scatter(res_key='leiden', base_image='/home/ryan/Sean/IMG/PigP10_crop/B03701E2_ssDNA_regist.tif', 
                         others_color='#FFFFFF', foreground_alpha=1,
                         groups=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
                                                  '12', '13', '14', '15', '16', '17', 
                                                   '18', '19', '20', '21', '22', '23'])#ignore off-tissue cluster (set to white by excluding from in-groups list)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/15_Spatial_Clustering_sub_wMask.png', dpi=500)

data.plt.spatial_scatter_by_gene(gene_name=['KRT14', 'KRT15'], palette='CET_L4', color_bar_reverse=False,
                                 out_dpi=500, height=500, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/Palette_Krt_CET_L4.png', dpi=500)

data.plt.spatial_scatter_by_gene(gene_name=['KRT14', 'KRT15'], palette='CET_L20', color_bar_reverse=False,
                                 out_dpi=500, height=500, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/Palette_KRT_CET_L20.png', dpi=500)

data.plt.spatial_scatter_by_gene(gene_name=['COL1A1', 'COL3A1'], palette='CET_L4', color_bar_reverse=False,
                                 out_dpi=500, height=500, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/Palette_COL_CET_L4.png', dpi=500)

data.plt.spatial_scatter_by_gene(gene_name=['COL1A1', 'COL3A1'], palette='CET_L20', color_bar_reverse=False,
                                 out_dpi=500, height=500, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/Palette_COL_CET_L20.png', dpi=500)

In [ ]:
#save the normalized StereoExpData to file
st.io.write_h5ad(data, use_raw=True, use_result=True, key_record=None, output='/home/ryan/Sean/Object/StereoSeq/PigP10_bin20_crop_normalized_v1.stereo.h5ad', split_batches=True)

In [ ]:
#convert the normalized StereoExpData object to be anndata (for Scanpy)
st.io.stereo_to_anndata(data, flavor='scanpy', sample_id='PigP10', output='/home/ryan/Sean/Object/StereoSeq/PigP10_bin20_crop_normalized_v1.stereo.forScanpy.h5ad')

In [ ]:
#load normalized stereo-seq data
data = st.io.read_h5ad(file_path='/home/ryan/Sean/Object/StereoSeq/PigP10_bin20_crop_normalized_v1.stereo.forScanpy.h5ad', spatial_key='spatial')#normalized
data#view stereopy object prepared from stereoseq
data.adata

In [ ]:
#convert the StereoExpData object to be anndata (for Seurat)
#st.io.stereo_to_anndata(data, sample_id='PigP10', flavor='seurat', output='/home/ryan/Sean/Object/StereoSeq/PigP10_bin20_crop_normalized_v1_stereo_forSeurat.h5ad')

In [ ]:
#convert the Scanpy flavor anndata object to save as adata
data.adata.write_h5ad('/home/ryan/Sean/Object/StereoSeq/PigP10_bin20_crop_normalized_v1_stereo_anndata.h5ad')

In [ ]:
#convert the normalized anndata.h5ad file in R to .rds using sceasy to read into a Seurat object SCTransform-normalized data

In [ ]:
#continue in stereopy with the normalized stereoexpdata object (load-in point)

#load normalized stereo-seq data
data = st.io.read_h5ad(file_path='/home/ryan/Sean/Object/StereoSeq/PigP10_bin20_crop_normalized_v1.stereo.h5ad', flavor='stereopy')#normalized
data#view stereopy object prepared from stereoseq

In [ ]:
#Find marker genes from spatial clusters
data.tl.find_marker_genes(
        cluster_res_key='leiden',
        method='t_test',
        use_highly_genes=False,
        use_raw=True
        )

In [ ]:
#view top markers of Stereo-seq Leiden clusters
data.plt.marker_genes_text(
        res_key='marker_genes',
        markers_num=10,
        sort_key='scores'
        )#Scatter of marker genes
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/16_MarkerGenes_Text.png', dpi=500)
data.plt.marker_genes_scatter(res_key='marker_genes', markers_num=10)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/16_MarkerGenes_Scatter.png', dpi=500)

In [ ]:
#filter marker genes
data.tl.filter_marker_genes(
    marker_genes_res_key='marker_genes',
    min_fold_change=1,
    min_in_group_fraction=0.1,
    max_out_group_fraction=0.5,
    res_key='marker_genes_filtered'
)

#view filtered top markers of Stereo-seq Leiden clusters
data.plt.marker_genes_text(
        res_key='marker_genes',
        markers_num=10,
        sort_key='scores'
        )#Scatter of marker genes
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/17_MarkerGenes_Text_filtered.png', dpi=500)
data.plt.marker_genes_scatter(res_key='marker_genes', markers_num=10)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/17_MarkerGenes_Scatter_filtered.png', dpi=500)

In [ ]:
#view subset of Leiden clustering onto the mask of the tissue section
leiden_clusters = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', 
                   '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', 
                   '21', '22', '23']

#prepare directory and pathing strings for autoexport
dir_out = '/home/ryan/Sean/PigP10/crop/Leiden_iso'#path to main output directory
#sample = 'PigP10'#sample id for output directory subfolder

#generate and export spatial_scatter visualizing each cluster individually
for i, cluster in enumerate(leiden_clusters):
    path_out = os.path.join(dir_out, cluster)#combine all pathing elements to get iterable file path

    #view spatial gene expression across the tissue mask
    data.plt.cluster_scatter(res_key='leiden', base_image='/home/ryan/Sean/IMG/PigP10_crop/B03701E2_ssDNA_regist.tif', 
                             others_color='#FFFFFF', foreground_alpha=1,
                             groups=leiden_clusters[i])#view each cluster one at a time
    plt.savefig(fname=path_out, dpi=500)
    
#data.plt.cluster_scatter(res_key='leiden', groups=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11'
#                                                  '12', '13', '14', '15', '16', '17', 
#                                                   '18', '19', '20', '21', '22', '23'], foreground_alpha=1)#exclude cluster 11 (not tissue)
#plt.savefig(fname='/home/ryan/Sean/PigP10/crop/8_Spatial_Clustering_focus.png', dpi=500)
#data.plt.cluster_scatter(res_key='leiden', groups=['16', '18'], base_image='/home/ryan/Sean/IMG/PigP10_crop/B03701E2_ssDNA_regist.tif',
#                        others_color='#FFFFFF', foreground_alpha=1)
#plt.savefig(fname='/home/ryan/Sean/PigP10/crop/8_filtered_Spatial_Clustering_focus_Epi_wMask.png', dpi=500)

In [ ]:
#view individual DEGs
data.plt.spatial_scatter_by_gene(gene_name=['CCL21'], palette='CET_L4', color_bar_reverse=False,
                                 out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/CCL21.png', dpi=500)
data.plt.spatial_scatter_by_gene(gene_name=['DLK2'], palette='CET_L4', color_bar_reverse=False,
                                 out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/DLK2.png', dpi=500)
data.plt.spatial_scatter_by_gene(gene_name=['CXCL14'], palette='CET_L4', color_bar_reverse=False,
                                 out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/CXCL14.png', dpi=500)
data.plt.spatial_scatter_by_gene(gene_name=['TGFBI'], palette='CET_L4', color_bar_reverse=False,
                                 out_dpi=500, height=1000, width=1000)
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/TGFBI.png', dpi=500)

In [ ]:
#rename clusters by cell identity
annotation_dict = {
    '1':'Fibroblast',
    '2':'Fibroblast',
    '3':'Vasculature',
    '4':'Adipocyte',
    '5':'Fibroblast',
    '6':'Fibroblast',
    '7':'HF_ORS',
    '8':'Perifollicular_Fibroblast',
    '9':'Papillary_Fibroblast',
    '10':'Fibroblast',
    '11':'Cornified_Envelope',
    '12':'HF_IRS',
    '13':'HF_Bulb',
    '14':'Sweat_Gland',
    '15':'Periglandular_Fibroblast',
    '16':'Differentiating_Keratinocyte',
    '17':'Lymphatic_Vessel',
    '18':'Basal_Keratinocyte',
    '19':'Fibroblast',
    '20':'HF_Cortex',
    '21':'Sweat_Gland_Duct',
    '22':'HF_Isthmus',
    '23':'Fibroblast'
}

In [ ]:
data.tl.annotation(
        annotation_information=annotation_dict,
        cluster_res_key='leiden',
        res_key='anno_leiden'
        )
data.plt.cluster_scatter(res_key='anno_leiden')

In [ ]:
#view spatial gene expression across the tissue mask
data.plt.cluster_scatter(res_key='leiden', base_image='/home/ryan/Sean/IMG/PigP10_crop/B03701E2_ssDNA_regist.tif', 
                         others_color='#FFFFFF', foreground_alpha=1,
                         groups=['16', '18'])#view key clusters
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/18_Spatial_Clustering_Epi_wMask.png', dpi=500)

data.plt.cluster_scatter(res_key='leiden', base_image='/home/ryan/Sean/IMG/PigP10_crop/B03701E2_ssDNA_regist.tif', 
                         others_color='#FFFFFF', foreground_alpha=1,
                         groups=['3', '9',
                                 '16', '17',
                                 '18'])#view key clusters
plt.savefig(fname='/home/ryan/Sean/PigP10/crop/18_Spatial_Clustering_EpiDerm_wMask.png', dpi=500)

In [ ]:
#save the normalized StereoExpData to file
st.io.write_h5ad(data, use_raw=True, use_result=True, key_record=None, output='/home/ryan/Sean/Object/StereoSeq/PigP10_bin20_crop_normalized_v2_labeled.stereo.h5ad', split_batches=True)

In [ ]:
#convert the normalized StereoExpData object to be anndata (for Scanpy)
st.io.stereo_to_anndata(data, flavor='scanpy', sample_id='PigP10', output='/home/ryan/Sean/Object/StereoSeq/PigP10_bin20_crop_normalized_v1_labeled.stereo.forScanpy.h5ad')